# Assignment 2: Voting Visualized

## Deadline

Oct. 24th

## Important notes

- Make sure you push on GitHub your notebook with all the cells already evaluated.
- Note that maps do not render in a standard Github environment. You should export them to HTML and link them in your notebook.
- Don't forget to add a textual description of your thought process, the assumptions you made, and the solution you implemented.
- Please write all your comments in English, and use meaningful variable names in your code.

## Background


* Are you curious to know what the political leanings of the people of Switzerland are?
* Do you wake up in a cold sweat, wondering which party won the last cantonal parliament election in Vaud?
* Are you looking to learn all sorts of visualizations, including maps, in Python?

If your answer to any of the above is yes, this assignment is just right for you. Otherwise, it's still an assignment, so we're terribly sorry.

The chief aim of this assignment is to familiarize you with visualizations in Python, particularly maps, and also to give you some insight into how visualizations are to be interpreted. The data we will use is the data on Swiss cantonal parliament elections from 2007 to 2018, which contains, for each cantonal election in this time period, the voting percentages for each party and canton.

For the visualization part, install [Folium](Folium) (_Hint: it is not available in your standard Anaconda environment, therefore search on the Web how to install it easily!_). Folium's README comes with very clear examples, and links to their own iPython Notebooks -- make good use of this information. For your own convenience, in this same directory you can already find one TopoJSON file, containing the geo-coordinates of the cantonal borders of Switzerland.

One last, general reminder: back up any hypotheses and claims with data, since this is an important aspect of the course.

In [1]:
import pandas as pd
import json
import folium
import os
import xlrd


In [2]:
data_folder = './data/'

# Task 1: Cartography and census

__A)__ Display a Swiss map that has cantonal borders as well as the national borders. We provide a TopoJSON `data/ch-cantons.topojson.json` that contains the borders of the cantons.

__B)__ Take the spreadsheet `data/communes_pop.xls`, collected from [admin.ch](https://www.bfs.admin.ch/bfs/fr/home/statistiques/catalogues-banques-donnees/tableaux.assetdetail.5886191.html), containing population figures for every commune. You can use [pd.read_excel()](https://pandas.pydata.org/pandas-docs/version/0.20/generated/pandas.read_excel.html) to read the file and to select specific sheets. Plot a histogram of the population counts and explain your observations. Do not use a log-scale plot for now. What does this histogram tell you about urban and rural communes in Switzerland? Are there any clear outliers on either side, and if so, which communes?

__C)__ The figure below represents 4 types of histogram. At this stage, our distribution should look like Fig.(a). A common way to represent [power-laws](https://en.wikipedia.org/wiki/Power_law) is to use a histogram using a log-log scale  -- remember: the x-axis of an histogram is segmented in bins of equal sizes and y-values are the average of each bin. As shown in Fig.(b), small bins sizes might introduce artifacts. Fig.(b) and Fig.(c) are examples of histograms with two different bin sizes. Another great way to visualize such distribution is to use a cumulative representation, as show in Fig.(d), in which the y-axis represents the number of data points with values greater than y.  
  
Create the figures (b) and (d) using the data extracted for task 1B. For Fig.(b), represent two histograms using two different bin sizes and provide a brief description of the results. What does this tell you about the relationship between the two variables, namely the frequency of each bin and the value (i.e. population in case of the communal data) for each bin?

<img src="plaw_crop.png" style="width: 600px;">
  
The figure is extracted from [this paper](https://arxiv.org/pdf/cond-mat/0412004.pdf) that contains more information about this family of distributions.

## Task 2: Parties visualized

We provide a spreadsheet, `data/voters.xls`, (again) collected from [admin.ch](https://www.bfs.admin.ch/bfs/fr/home/statistiques/politique/elections/conseil-national/force-partis.assetdetail.217195.html), which contains the percentage of voters for each party and for each canton. For the following task, we will focus on the period 2014-2018 (the first page of the spreadsheet). Please report any assumptions you make regarding outliers, missing values, etc. Notice that data is missing for two cantons, namely Appenzell Ausserrhoden and Graubünden, and your visualisations should include data for every other canton.


__A)__ For the period 2014-2018 and for each canton, visualize, on the map, **the percentage of voters** in that canton who voted for the party [`UDC`](https://en.wikipedia.org/wiki/Swiss_People%27s_Party) (Union démocratique du centre). Does this party seem to be more popular in the German-speaking part, the French-speaking part, or the Italian-speaking part?

__B)__ For the same period, now visualize **the number of residents** in each canton who voted for UDC.

__C)__ Which one of the two visualizations above would be more informative in case of a national election with majority voting (i.e. when a party needs to have the largest number of citizens voting for it among all parties)? Which one is more informative for the cantonal parliament elections?

For part B, you can use the `data/national_council_elections.xslx` file ([guess where we got it from](https://www.bfs.admin.ch/bfs/fr/home/statistiques/politique/elections/conseil-national/participation.assetdetail.81625.html)) to have the voting-eligible population of each canton in 2015.

In [13]:
voters = pd.read_excel(data_folder + 'voters.xls')
voters

,"Elections des parlements cantonaux, de 2014 à 2018: force des partis et attribution des listes mixtes* aux partis",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,T 17.02.05.02.03
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,Année électorale 2),Participation,PLR 6),NaN,PDC 7),NaN,PS,NaN,...,JB,NaN,Front,NaN,Grut,NaN,Autres 11),NaN,K,Total
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Zurich,NaN,2015,32.6525,17.3278,NaN,4.87871,NaN,19.7164,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.669707,NaN,NaN,100
6,Berne,NaN,2018,30.5163,11.7179,NaN,0.671415,NaN,22.3288,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.912781,NaN,NaN,100
7,Lucerne,NaN,2015,38.7413,21.0395,NaN,30.8625,NaN,11.8489,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0361293,NaN,NaN,100
8,Uri 1),NaN,2016,61.9891,26.8567,NaN,31.2988,NaN,12.985,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.42827,NaN,NaN,100
9,Schwytz,NaN,2016,37.7471,21.629,NaN,27.1677,NaN,12.9254,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.51358,NaN,NaN,100


In [4]:
#we load the voters dataset
voters = pd.read_excel(data_folder + 'voters.xls')

#we load the cantons dataset
cantons = pd.read_csv(data_folder + 'cantons.csv')

#Cleaning of the dataset

#First we want to drop all the rows full of NaN
#Creation of a new df
voters_noNaN = voters.drop(voters.index[0:3])

#We drop the first column in order to have row full of NaN and be able to use classic pandas functions 
voters_noNaN = voters_noNaN.drop(voters_noNaN.columns[0],axis=1)

#drop the NaN rows...
voters_noNaN=voters_noNaN.dropna(axis=0,how='all')

#We keep the column index of or original dataframe that corresponds to the one of the non-NaN df
voters=voters.iloc[voters_noNaN.index]

#Now we can drop the columns full of NaN
voters=voters.dropna(axis=1,how='all')

#The rest of NaN values corresponds to state where the political part is not present i.e. there was 0 voters so we replace NaN by null
voters=voters.fillna(0)

#We rename the columns with corresponding labels
voters.columns = ['Cantons', 'Année électorale', 'Participation', 'PLR', 'PDC', 'PS','UDC','PLS','PEV','PCS','PVL','PBD','PST','PSA','PES','AVF','Sol.','DS','UDF','Lega','MCR','Autres','Total']

#Reset in the index in order to be able to concatenate our dataframe with cantons dataframe
voters.reset_index(drop=True,inplace =True)

#Concatenation
left = voters
right = cantons
left.join(right,sort = False)
result=pd.concat([left,right],axis=1,sort=False)

#We convert the population data into float in order to be able to do operations on it
result['Population']= result['Population'].apply(lambda row: float(row.replace(",","")[:-4]))

#Since the column 'Canton of' is redundant we use it to create a new columns equal to the number of UDC voters
result['Canton of'] = result['Population']*result['Participation']*result['UDC']*0.0001
result.rename(columns={'Canton of':'UDC voters'}, inplace=True)

#Creation of CSV file to use with folium
result.to_csv('result.csv')
result_csv = r'result.csv'
result_data=pd.read_csv(result_csv)
result_data.drop(result_data.columns[0],axis=1,inplace=True)


result

,Cantons,Année électorale,Participation,PLR,PDC,PS,UDC,PLS,PEV,PCS,...,Code,UDC voters,Since,Capital,Population,GDP percapita (2014)[16]in CHF,Area (km2),Density (per km2)[Note 4],No. munic.,Official languages
0,Zurich,2015,32.652497,17.327820,4.878712,19.716444,30.023215,0.000000,4.271767,0.000000,...,ZH,147475.993602,1351,Zürich,1504346.0,"96,411","1,729",701,168,German
1,Berne,2018,30.516313,11.717910,0.671415,22.328775,26.760869,0.000000,6.172897,0.000000,...,BE,84206.189136,1353,Bern,1031126.0,"76,307","5,960",158,352,"German, French"
2,Lucerne,2015,38.741267,21.039516,30.862513,11.848915,24.115646,0.000000,0.199143,0.000000,...,LU,37688.198914,1332,Lucerne,403397.0,"65,119","1,494",233,83,German
3,Uri 1),2016,61.989056,26.856698,31.298814,12.984966,24.053191,0.000000,0.000000,0.000000,...,UR,5389.344461,1291,Altdorf,36145.0,"51,332","1,077",33,20,German
4,Schwytz,2016,37.747088,21.628996,27.167707,12.925363,33.115090,0.000000,0.304428,0.000000,...,SZ,19482.846935,1291,Schwyz,155863.0,"58,788",908,143,30,German
5,Obwald,2018,53.793314,17.174703,29.803630,15.090867,24.532303,0.000000,0.000000,0.000000,...,OW,4932.677027,1291,Sarnen,37378.0,"64,253",491,66,7,German
6,Nidwald,2018,54.921606,28.012430,26.750081,4.470053,25.916631,0.000000,0.000000,0.000000,...,NW,6057.348544,1291,Stans,42556.0,"69,559",276,138,11,German
7,Glaris,2018,29.489675,18.392812,9.400847,12.750408,25.275361,0.000000,0.000000,0.000000,...,GL,3007.461872,1352,Glarus,40349.0,"67,379",685,51,3,German
8,Zoug,2014,42.939437,22.147887,26.783136,9.250912,23.631802,0.000000,0.000000,0.000000,...,ZG,12577.453332,1352,Zug,123948.0,"150,613",239,416,11,German
9,Fribourg,2016,39.302102,18.165508,23.707027,23.584309,19.717606,0.000000,0.000000,3.644715,...,FR,24416.450459,1481,Fribourg,315074.0,"58,369","1,671",141,150,"French, German"


In [5]:
#Data visualization for UDC vote percentage by canton
cantons_data= json.load(open(data_folder + 'ch-cantons.topojson.json'))

m = folium.Map([47, 8.33],tiles='cartodbpositron', zoom_start=7)




m.choropleth(
 geo_data=cantons_data,
 topojson='objects.cantons',   
 name='choropleth',
 data=result_data,
 columns=['Code', 'UDC'],
 key_on='feature.id',
 fill_color='YlGn',
 fill_opacity=0.7,
 line_opacity=0.2,
 legend_name='UDC vote Rate (%)'
)
folium.LayerControl().add_to(m)

m.save('UDC vote rate.html')
m

In [6]:
#Data visualization for number of UDC voters by canton

m = folium.Map([47, 8.33],tiles='cartodbpositron', zoom_start=7)


m.choropleth(
 geo_data=cantons_data,
 topojson='objects.cantons',   
 name='choropleth',
 data=result,
 columns=['Code', 'UDC voters'],
 key_on='feature.id',
 fill_color='YlGn',
 fill_opacity=0.7,
 line_opacity=0.2,
 legend_name='Number of UDC voters',
 threshold_scale=[0,10000,30000,60000, 90000,120000]
)
folium.LayerControl().add_to(m)
m.save('UDC voters number.html')
m

In [7]:
#Does this party seem to be more popular in the German-speaking part, the French-speaking part, or the Italian-speaking part?
#ATTENTION J'AI EU BCP DE MAL A FAIRE CETTE MAP QUI N'EST PAS DEMANDEE ! DU COUP C'EST PAS OPTI


german_part = result['Official languages'] == 'German'
french_part = result['Official languages'] == 'French'
italian_part = result['Official languages'] == 'Italian'

result['Population']=result['Participation']*result['Population']/100

german_swiss_UDC_popularity=result[german_part]['UDC voters'].sum()/result[german_part]['Population'].sum()
french_swiss_UDC_popularity=result[french_part]['UDC voters'].sum()/result[french_part]['Population'].sum()
italian_swiss_UDC_popularity=result[italian_part]['UDC voters'].sum()/result[french_part]['Population'].sum()

result_french=result[french_part]

result_german=result[german_part]

result_italian=result[italian_part]
result

,Cantons,Année électorale,Participation,PLR,PDC,PS,UDC,PLS,PEV,PCS,...,Code,UDC voters,Since,Capital,Population,GDP percapita (2014)[16]in CHF,Area (km2),Density (per km2)[Note 4],No. munic.,Official languages
0,Zurich,2015,32.652497,17.327820,4.878712,19.716444,30.023215,0.000000,4.271767,0.000000,...,ZH,147475.993602,1351,Zürich,491206.535647,"96,411","1,729",701,168,German
1,Berne,2018,30.516313,11.717910,0.671415,22.328775,26.760869,0.000000,6.172897,0.000000,...,BE,84206.189136,1353,Bern,314661.635440,"76,307","5,960",158,352,"German, French"
2,Lucerne,2015,38.741267,21.039516,30.862513,11.848915,24.115646,0.000000,0.199143,0.000000,...,LU,37688.198914,1332,Lucerne,156281.107399,"65,119","1,494",233,83,German
3,Uri 1),2016,61.989056,26.856698,31.298814,12.984966,24.053191,0.000000,0.000000,0.000000,...,UR,5389.344461,1291,Altdorf,22405.944159,"51,332","1,077",33,20,German
4,Schwytz,2016,37.747088,21.628996,27.167707,12.925363,33.115090,0.000000,0.304428,0.000000,...,SZ,19482.846935,1291,Schwyz,58833.743000,"58,788",908,143,30,German
5,Obwald,2018,53.793314,17.174703,29.803630,15.090867,24.532303,0.000000,0.000000,0.000000,...,OW,4932.677027,1291,Sarnen,20106.864906,"64,253",491,66,7,German
6,Nidwald,2018,54.921606,28.012430,26.750081,4.470053,25.916631,0.000000,0.000000,0.000000,...,NW,6057.348544,1291,Stans,23372.438488,"69,559",276,138,11,German
7,Glaris,2018,29.489675,18.392812,9.400847,12.750408,25.275361,0.000000,0.000000,0.000000,...,GL,3007.461872,1352,Glarus,11898.789038,"67,379",685,51,3,German
8,Zoug,2014,42.939437,22.147887,26.783136,9.250912,23.631802,0.000000,0.000000,0.000000,...,ZG,12577.453332,1352,Zug,53222.573963,"150,613",239,416,11,German
9,Fribourg,2016,39.302102,18.165508,23.707027,23.584309,19.717606,0.000000,0.000000,3.644715,...,FR,24416.450459,1481,Fribourg,123830.704396,"58,369","1,671",141,150,"French, German"


In [8]:
result_french.loc[:,'Participation']=french_swiss_UDC_popularity
result_german.loc[:,'Participation']=german_swiss_UDC_popularity
result_italian.loc[:,'Participation']=italian_swiss_UDC_popularity

print(french_swiss_UDC_popularity)
print(german_swiss_UDC_popularity)
print(italian_swiss_UDC_popularity)

0.12473204358828786
0.27814339707531344
0.016114799374631734


C:\Users\Ruijia\Anaconda2\envs\ada\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [9]:
frames = [result_french,result_german,result_italian]
frenchvsgermanpopularity=pd.concat(frames)
frenchvsgermanpopularity

,Cantons,Année électorale,Participation,PLR,PDC,PS,UDC,PLS,PEV,PCS,...,Code,UDC voters,Since,Capital,Population,GDP percapita (2014)[16]in CHF,Area (km2),Density (per km2)[Note 4],No. munic.,Official languages
21,Vaud,2017,0.124732,33.068194,2.213533,23.438908,15.842675,0.000000,0.331699,0.00000,...,VD,48397.634716,1803,Lausanne,305489.031860,"68,084","3,212",188,318,French
23,Neuchâtel,2017,0.124732,33.353683,2.680728,23.582051,11.489286,0.000000,0.000000,0.00000,...,NE,6860.846263,1815/1857,Neuchâtel,59715.168369,"83,835",802,206,31,French
24,Genève,2018,0.124732,25.183926,10.712970,15.299358,7.315683,0.000000,0.000000,0.00000,...,GE,13508.096227,1815,Geneva,184645.720844,"102,113",282,"1,442",45,French
25,Jura,2015,0.124732,15.433764,26.784772,20.047550,11.980336,0.000000,0.000000,12.12648,...,JU,4440.293936,1979,Delémont,37063.185103,"64,606",839,82,57,French
0,Zurich,2015,0.278143,17.327820,4.878712,19.716444,30.023215,0.000000,4.271767,0.00000,...,ZH,147475.993602,1351,Zürich,491206.535647,"96,411","1,729",701,168,German
2,Lucerne,2015,0.278143,21.039516,30.862513,11.848915,24.115646,0.000000,0.199143,0.00000,...,LU,37688.198914,1332,Lucerne,156281.107399,"65,119","1,494",233,83,German
3,Uri 1),2016,0.278143,26.856698,31.298814,12.984966,24.053191,0.000000,0.000000,0.00000,...,UR,5389.344461,1291,Altdorf,22405.944159,"51,332","1,077",33,20,German
4,Schwytz,2016,0.278143,21.628996,27.167707,12.925363,33.115090,0.000000,0.304428,0.00000,...,SZ,19482.846935,1291,Schwyz,58833.743000,"58,788",908,143,30,German
5,Obwald,2018,0.278143,17.174703,29.803630,15.090867,24.532303,0.000000,0.000000,0.00000,...,OW,4932.677027,1291,Sarnen,20106.864906,"64,253",491,66,7,German
6,Nidwald,2018,0.278143,28.012430,26.750081,4.470053,25.916631,0.000000,0.000000,0.00000,...,NW,6057.348544,1291,Stans,23372.438488,"69,559",276,138,11,German


In [10]:
result['Participation'].iloc[frenchvsgermanpopularity.index]=frenchvsgermanpopularity['Participation']*100

result.loc [[1,9,22],['Participation']]= 0.0
result

C:\Users\Ruijia\Anaconda2\envs\ada\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,Cantons,Année électorale,Participation,PLR,PDC,PS,UDC,PLS,PEV,PCS,...,Code,UDC voters,Since,Capital,Population,GDP percapita (2014)[16]in CHF,Area (km2),Density (per km2)[Note 4],No. munic.,Official languages
0,Zurich,2015,27.814340,17.327820,4.878712,19.716444,30.023215,0.000000,4.271767,0.000000,...,ZH,147475.993602,1351,Zürich,491206.535647,"96,411","1,729",701,168,German
1,Berne,2018,0.000000,11.717910,0.671415,22.328775,26.760869,0.000000,6.172897,0.000000,...,BE,84206.189136,1353,Bern,314661.635440,"76,307","5,960",158,352,"German, French"
2,Lucerne,2015,27.814340,21.039516,30.862513,11.848915,24.115646,0.000000,0.199143,0.000000,...,LU,37688.198914,1332,Lucerne,156281.107399,"65,119","1,494",233,83,German
3,Uri 1),2016,27.814340,26.856698,31.298814,12.984966,24.053191,0.000000,0.000000,0.000000,...,UR,5389.344461,1291,Altdorf,22405.944159,"51,332","1,077",33,20,German
4,Schwytz,2016,27.814340,21.628996,27.167707,12.925363,33.115090,0.000000,0.304428,0.000000,...,SZ,19482.846935,1291,Schwyz,58833.743000,"58,788",908,143,30,German
5,Obwald,2018,27.814340,17.174703,29.803630,15.090867,24.532303,0.000000,0.000000,0.000000,...,OW,4932.677027,1291,Sarnen,20106.864906,"64,253",491,66,7,German
6,Nidwald,2018,27.814340,28.012430,26.750081,4.470053,25.916631,0.000000,0.000000,0.000000,...,NW,6057.348544,1291,Stans,23372.438488,"69,559",276,138,11,German
7,Glaris,2018,27.814340,18.392812,9.400847,12.750408,25.275361,0.000000,0.000000,0.000000,...,GL,3007.461872,1352,Glarus,11898.789038,"67,379",685,51,3,German
8,Zoug,2014,27.814340,22.147887,26.783136,9.250912,23.631802,0.000000,0.000000,0.000000,...,ZG,12577.453332,1352,Zug,53222.573963,"150,613",239,416,11,German
9,Fribourg,2016,0.000000,18.165508,23.707027,23.584309,19.717606,0.000000,0.000000,3.644715,...,FR,24416.450459,1481,Fribourg,123830.704396,"58,369","1,671",141,150,"French, German"


In [12]:
m = folium.Map([47, 8.33],tiles='cartodbpositron', zoom_start=7)



m.choropleth(
 geo_data=cantons_data,
 topojson='objects.cantons',   
 name='choropleth',
 data=result,
 columns=['Code', 'Participation'],
 key_on='feature.id',
 fill_color='YlGn',
 fill_opacity=0.7,
 line_opacity=0.1,
 legend_name='UDC vote Rate in german and french speaking Switzerland(%)',
 threshold_scale=[0,5,10,15,20,25]
 
)
folium.LayerControl().add_to(m)
m.save('UDC vote rate GERvsFR.html')
m

Which one of the two visualizations above would be more informative in case of a national election with majority voting (i.e. when a party needs to have the largest number of citizens voting for it among all parties)? Which one is more informative for the cantonal parlement elections?

In case of a national election with majority voting it is clearly the visualization of part B with the raw voters number that is the more usefull. The only problem is that the reader need to have an idea of the voters total population in order to understand the numbers on this map. It is good to be able to see that UDC can have 30 000 voters in one canton but if the PS have 50 000 UDC will not win despite the impressive number... Thus, percentage is often more clear unlike my explanation.

On the other hand, for the cantonal parlement elections, it is the popularity of the party by cantons that will be important and the visualization of part A.